<a href="https://colab.research.google.com/github/LUCY-JungHwaKim/DataAnalysis/blob/main/Part2_chap5_N2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 19.96 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [2]:
import pandas as pd

wine = pd.read_csv('./yemoonsaBigdata/datasets/Part2/wine.csv')
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,NaN,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [3]:
##문제 1. total_phenols 결측치 제거, alcalinity_of_ash 결측치 중앙값으로 대체 후 alcalinity_of_ash 평균값 계산 (최종 계산값의 소수점 이하는 버림)
import numpy as np
wine_noph = wine[~wine.total_phenols.isna()] #total_phenols 결측치 제거
wine_noph['alcalinity_of_ash'] = np.where(wine_noph.alcalinity_of_ash.isna(), wine_noph.alcalinity_of_ash.median(), wine_noph.alcalinity_of_ash)
int(wine_noph.alcalinity_of_ash.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


19

In [6]:
# 문제 2. alchol값이alcohol 평균값 이상이고
# color_intensity 값이 color_intensity 평균값 이상인 데이터만 남겼을 때
# 가장 많은 class 값 구하기
alch_avg = wine.alcohol.mean()
col_avg = wine.color_intensity.mean()
wine_filt = wine[(wine.alcohol >= alch_avg) & (wine.color_intensity >= col_avg)]
wine_filt['class'].value_counts()
##Ans) 0

0    37
2    23
1     1
Name: class, dtype: int64

In [44]:
# 문제 3. 클래스 1에 해당하는 데이터의 ash 평균값
# wine 데이터셋의 모든 결측치를 제거한 후의 ash 평균값
# 이둘의 차이의 절대값 계싼 (소수점 아래 4번째 자리에서 반올림)
wine_cl1 = wine[wine['class'] == 1] 
wine_cl1_avg = wine_cl1.ash.mean()
wine_nona = wine[(~wine.alcalinity_of_ash.isna()) & (~wine.total_phenols.isna())] # 모든 결측치 제거한 후의 데이터셋
wine_nona_avg = wine_nona.ash.mean()
round(abs(wine_cl1_avg - wine_nona_avg), 3) # 평균의차이를 절대값한 후 반올림 - 4번째 자리에서 반올림한다고 했으니깐 round(, 3)

0.12

In [48]:
## 문제 4번.
wine_assort = wine.sort_values('alcohol', ascending=True) ## alcohol을 오름차순으로 정렬
top20_alco = wine_assort.iloc[19:20].alcohol ## 20번째 위치한 alcohol값  (float 안해도 됨)
wine_assort
wine_assort.iloc[:20].alcohol = [top20_alco for i in range(20)] # 상위 20개의 alcohol값을 위에서 추출한 값으로 변환
pro_filt = wine_assort[wine_assort.proanthocyanins <= 2] # 다음 필터링 기준을 갖는 데이터 추출해서 alcohol의 평균값 계산
round(pro_filt.alcohol.mean(), 2) # 최종 계산값의 소수점 아래 3번째 ㅐ자리에서 반올림 -> round(, 2)
#round 사용은 결과값과 문제에서 원하는 값을 보고 round 인자 조정하기

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


12.94

In [50]:
wine_assort.iloc[19:20].alcohol

119    12.0
Name: alcohol, dtype: float64

In [42]:
#문제 5번. class1에 해당하는 데이터의 proline 변수의 제3사분위수와 제1사분위수 차를 구하고
# class2에 해당하는 데이터도 마찬가지.
# 두 값의 차이의 절대값 구하기
cl1_pro = wine_cl1.proline.describe()[6] - wine_cl1.proline.describe()[4]
wine_cl2 = wine[wine['class'] == 2]
cl2_pro = wine_cl2.proline.describe()[6] - wine_cl2.proline.describe()[4]
abs(cl1_pro - cl2_pro)

68.5